In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.nn.functional import normalize
import time
import pickle
from datetime import timedelta
np.random.seed(6488)

X_train_QRT = pd.read_csv('./X_train.csv', index_col=0, sep=',')
X_train_QRT.columns.name = 'date'

Y_train_QRT = pd.read_csv('./Y_train.csv', index_col=0, sep=',')
Y_train_QRT.columns.name = 'date'

X_train_QRT_reshape = pd.concat([ X_train_QRT.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_QRT_reshape.columns = pd.Index(range(1,251), name='timeLag')

N0 = 3

def split(j):
            perm = np.random.permutation(range(50))
            valset = perm[:j]
            trainset = perm[j:]
            X_1 = X_train_QRT.iloc[valset,]
            X_2 = X_train_QRT.iloc[trainset,]
            Y_1 = Y_train_QRT.iloc[valset,]
            Y_2 = Y_train_QRT.iloc[trainset,]
            return X_2,Y_2,X_1,Y_1,trainset

X_train_Q,Y_train_Q,X_test,Y_test,test_locations = split(N0)
        # Here we define our train and validation sets ! 
    
X_test.columns.name = 'date' 

X_test_reshape = pd.concat([ X_test.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_test_reshape.columns = pd.Index(range(1,251), name='timeLag')

#X_train_Q = pd.read_csv('./X_train.csv', index_col=0, sep=',')
#X_train_Q.columns.name = 'date'

#Y_train_Q = pd.read_csv('./Y_train.csv', index_col=0, sep=',')
#Y_train_Q.columns.name = 'date'

X_train_Q_reshape = pd.concat([ X_train_Q.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_Q_reshape.columns = pd.Index(range(1,251), name='timeLag')

def randomA(D=250, F=10):  

    M = np.random.randn(D,F)
    randomStiefel = np.linalg.qr(M)[0] # Apply Gram-Schmidt algorithm to the columns of M

    return randomStiefel

def checkOrthonormality(A): 

        bool = True
        D, F = A.shape   
        Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()

        if any(Error.unstack() > 1e-6):
            bool = False

        return bool
    
def parametersTransform(A, beta, D=250, F=10):
    
    if A.shape != (D, F):
        print('A has not the good shape')
        return
    
    if beta.shape[0] != F:
        print('beta has not the good shape')
        return        
    
    output = np.hstack( (np.hstack([A.T, beta.reshape((F, 1))])).T )
    
    return output

def fitBeta_QRT(A):

            predictors = X_train_Q_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
            targets = Y_train_Q.T.stack()
            beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets

            return beta.to_numpy()
        
def metric_QRT(A, beta): # Attention, on évalue la métrique sur X_train, l'échantillon sélectionné !! 

            if not checkOrthonormality(A):
                return -1.0    

            Ypred = (X_train_Q_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice (50-n)*504. Les vecteurs de Ypred sont les prédictions !!      
            Ytrue = Y_train_Q

            Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
            Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

            meanOverlap = (Ytrue * Ypred).sum().mean()

            return  meanOverlap  
        
def metric_test(A, beta): # On évalue la métrique sur X_test et Y_test définis au tout début, et auxquels le modèle n'a jamais eu accès.  

            if not checkOrthonormality(A):
                return -1.0    

            Ypred = (X_test_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice (50-n)*504. Les vecteurs de Ypred sont les prédictions !!      
            Ytrue = Y_test

            Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
            Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

            meanOverlap = (Ytrue * Ypred).sum().mean()

            return  meanOverlap  
        
def metric_QRT_bch(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X_train_QRT_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice 50*504. Les vecteurs de Ypred sont les prédictions !!      
    Ytrue = Y_train_QRT
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap  



def Cross_FV(n=4,Niter = 100):
    
    max_accur = 0
    
    for l in range(Niter):
        
        
    
        def My_Own_CF(j):
            perm = np.random.permutation(range(50-N0))
            valset = perm[:j]
            trainset = perm[j:]
            X_1 = X_train_Q.iloc[valset,]
            X_2 = X_train_Q.iloc[trainset,]
            Y_1 = Y_train_Q.iloc[valset,]
            Y_2 = Y_train_Q.iloc[trainset,]
            return X_2,Y_2,X_1,Y_1,valset

        X_train,Y_train,X_val,Y_val,val_locations = My_Own_CF(n)
        # Here we define our train and validation sets ! 


        X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
        X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

        X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
        X_val_reshape.columns = pd.Index(range(1,251), name='timeLag')

        # Tout le bazar stdt, pas besoin de se le retaper à chaque fois
        targets_t = torch.tensor(Y_train.T.stack().to_numpy().astype(np.float32)) # Le gros vecteur de pred 
        Y_true_t = torch.tensor(Y_train.to_numpy().astype(np.float32)) # La matrice à prédire en torch
        targets_val_t = torch.tensor(Y_val.T.stack().to_numpy().astype(np.float32)) # Le gros vecteur de pred du validation set  
        Y_true_val_t = torch.tensor(Y_val.to_numpy().astype(np.float32)) # La matrice à prédire du validation set en torch
        X_train_reshape_t = torch.tensor(X_train_reshape.to_numpy().astype(np.float32)) # X_train_reshape en torch
        X_val_reshape_t = torch.tensor(X_val_reshape.to_numpy().astype(np.float32)) # X_val_reshape en torch

        def fitBeta_train(A):

            predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
            targets = Y_train.T.stack()
            beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets

            return beta.to_numpy()
        
        def fitBeta_val(A): # A première vue on n'en a pas besoin, mais bon.  
            
            predictors = X_val_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
            targets = Y_val.T.stack()
            beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets

            return beta.to_numpy()

        def metric_train(A, beta): # Attention, on évalue la métrique sur X_train, l'échantillon sélectionné !! 

            if not checkOrthonormality(A):
                return -1.0    

            Ypred = (X_train_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice (50-n)*504. Les vecteurs de Ypred sont les prédictions !!      
            Ytrue = Y_train

            Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
            Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

            meanOverlap = (Ytrue * Ypred).sum().mean()

            return  meanOverlap  

        def metric_val(A, beta): # On évalue la métrique sur le complémentaire 

            if not checkOrthonormality(A):
                return -1.0    

            Ypred = (X_val_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice n*504. Les vecteurs de Ypred sont les prédictions du validation set!!      
            Ytrue = Y_val

            Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
            Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

            meanOverlap = (Ytrue * Ypred).sum().mean()

            return  meanOverlap  

        #On calcule en même temps la fonction à maximiser (sortie 1) et son gradient au point A (sortie 2) sur le training set !! Pour ne pas changer le bordel dans algo_curv_search (notations etc...), 
        # on définit cette fonction à l'intérieur de algo_curv_search ! 
        def F_et_gradient_train(A):

            A_t = torch.tensor(A.astype(np.float32),requires_grad = True) # On transforme l'entrée en tensor (on spécifie qu'on va différencier par rapport à A_t !!)
            predictors = X_train_reshape_t @ A_t # Les prédicteurs en tensor 
            beta_t = torch.inverse(predictors.T @ predictors)@predictors.T @targets_t # beta de l'équation normale en tensor
            Y_pred_t = (X_train_reshape_t @ A_t @ beta_t).resize(504,(50-N0-n)).T #La matrice de prédiction en tensor
            f_A = torch.mean(sum(normalize(Y_pred_t,dim=0)*normalize(Y_true_t,dim=0)))
            f_A.backward() # On calcule le gradient
            A_output = -1*(A_t.grad).numpy() # Et voici le gradient en A !! C'est magique tellement c'est facile... 
            func_val = -1*f_A.detach().numpy() #la fonction de coût à minimiser évaluée en A 

            return func_val, A_output

        def algo_curv_search_train(X_start = randomA(),rho1 = 0.000001,rho2 = 0.5,eps = 0.001,j = 40): # la descente de gradient sur X_train
            start = time.time()
            X = X_start
            i,I = 0,np.eye(250)
            f,G = F_et_gradient_train(X) # f is f(X) et G the gradient of f at point X 
            tau_min = 0.0000001
            tau_max = 10
            #runn_scores = []

            while  (np.linalg.norm(G) >= eps) and (i <= j):
                i += 1
                #if i % 5 == 0:
                    #print("Nombre d'itérations:",i,"score actuel:", -f,"gradient actuel", np.linalg.norm(G))
                    #print(time.ctime(time.time() - start)) # to keep track of the evolution of the algorithm
                tau_min = 0.000001
                tau_max = 10
                W = G @ X.T - X @G.T #the skew symmetric matrix of the article _ we try to stick to the notations !  Remember, we need to update these at the end of the Armijo Wolfe conditions.          
                DF_0 = np.trace(G.T @ W @ X) #the directional derivative F'(Y(0)). 
                M_inv_min = np.linalg.inv(I + tau_min/2 * W) # Useful inverse matrices which we do not want to compute twice
                M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                f_t_max,G_t_max = F_et_gradient_train(Y_t_max)
                f_t_min,G_t_min = F_et_gradient_train(Y_t_min)
                cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max
                while (not cond_min) or (not cond_max) :
                    if not cond_min:
                        #print("Boucle d'itération:",i,"là c'est cond_min:", tau_min,tau_max) #pour check
                        if tau_min < 0.01:
                            tau_min = min(10*tau_min,tau_max - 0.0000000001)
                        else :
                            tau_min = min(1.05 * tau_min,tau_max - 0.000000001) 
                        #tau_min = .5*(tau_min + tau_max) # dichotomy is way too violent in this case... 
                        M_inv_min = np.linalg.inv(I + tau_min/2 * W)
                        Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                        f_t_min,G_t_min = F_et_gradient_train(Y_t_min)
                        cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                    elif not cond_max:
                        #print("Boucle d'itération:",i,"là c'est cond_max:", tau_min,tau_max) #pour check
                        tau_max = max(tau_min+ 0.000000001,min(.95*tau_max,.5*(tau_max+tau_min))) # just to ensure that we always have tau_max > tau_min
                        # tau_max = .5*(tau_min + tau_max) same comment
                        M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                        Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                        f_t_max,G_t_max = F_et_gradient_train(Y_t_max)
                        cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max

                tau_f = .5*(tau_min + tau_max) # t_k satisfying both AW conditions
                Y_t_f = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f_t_f,G_t_f = F_et_gradient_train(Y_t_f)
                #cond_f_1 = (f_t_f <= f + rho1 * tau_f * DF_0) #for control
                #cond_f_2 = (.5 * np.trace(G_t_f.T @ np.linalg.inv(I + tau_f/2 * W) @ W @ (X + Y_t_f)) >= rho2* DF_0) #for control
                #print("Boucle d'itération:",i,"un tau qui va", tau_f, cond_f_1, cond_f_2 ) #for control 
                X = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f,G = F_et_gradient_train(X)
                #runn_scores.append(f)
                #if npmean(runn_scores[::-20])

            #print("Le score final est:",-f)
            return X,-f
        
        
        def F_et_gradient_val(A):

            A_t = torch.tensor(A.astype(np.float32),requires_grad = True) # On transforme l'entrée en tensor (on spécifie qu'on va différencier par rapport à A_t !!)
            predictors = X_val_reshape_t @ A_t # Les prédicteurs en tensor 
            beta_t = torch.inverse(predictors.T @ predictors)@predictors.T @targets_val_t # beta de l'équation normale en tensor
            Y_pred_t = (X_val_reshape_t @ A_t @ beta_t).resize(504,n).T #La matrice de prédiction en tensor
            f_A = torch.mean(sum(normalize(Y_pred_t,dim=0)*normalize(Y_true_val_t,dim=0)))
            f_A.backward() # On calcule le gradient
            A_output = -1*(A_t.grad).numpy() # Et voici le gradient en A !! C'est magique tellement c'est facile... 
            func_val = -1*f_A.detach().numpy() #la fonction de coût à minimiser évaluée en A 

            return func_val, A_output

        def algo_curv_search_val(X_start = randomA(),rho1 = 0.000001,rho2 = 0.5,eps = 0.001,j = 40): # la descente de gradient sur X_train
            start = time.time()
            X = X_start
            i,I = 0,np.eye(250)
            f,G = F_et_gradient_val(X) # f is f(X) et G the gradient of f at point X 
            tau_min = 0.0000001
            tau_max = 10
            #runn_scores = []

            while  (np.linalg.norm(G) >= eps) and (i <= j):
                i += 1
                #if i % 5 == 0:
                    #print("Nombre d'itérations:",i,"score actuel:", -f,"gradient actuel", np.linalg.norm(G))
                    #print(time.ctime(time.time() - start)) # to keep track of the evolution of the algorithm
                tau_min = 0.000001
                tau_max = 10
                W = G @ X.T - X @G.T #the skew symmetric matrix of the article _ we try to stick to the notations !  Remember, we need to update these at the end of the Armijo Wolfe conditions.          
                DF_0 = np.trace(G.T @ W @ X) #the directional derivative F'(Y(0)). 
                M_inv_min = np.linalg.inv(I + tau_min/2 * W) # Useful inverse matrices which we do not want to compute twice
                M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                f_t_max,G_t_max = F_et_gradient_val(Y_t_max)
                f_t_min,G_t_min = F_et_gradient_val(Y_t_min)
                cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max
                while (not cond_min) or (not cond_max) :
                    if not cond_min:
                        #print("Boucle d'itération:",i,"là c'est cond_min:", tau_min,tau_max) #pour check
                        if tau_min < 0.01:
                            tau_min = min(10*tau_min,tau_max - 0.0000000001)
                        else :
                            tau_min = min(1.05 * tau_min,tau_max - 0.000000001) 
                        #tau_min = .5*(tau_min + tau_max) # dichotomy is way too violent in this case... 
                        M_inv_min = np.linalg.inv(I + tau_min/2 * W)
                        Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                        f_t_min,G_t_min = F_et_gradient_val(Y_t_min)
                        cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                    elif not cond_max:
                        #print("Boucle d'itération:",i,"là c'est cond_max:", tau_min,tau_max) #pour check
                        tau_max = max(tau_min+ 0.000000001,min(.95*tau_max,.5*(tau_max+tau_min))) # just to ensure that we always have tau_max > tau_min
                        # tau_max = .5*(tau_min + tau_max) same comment
                        M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                        Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                        f_t_max,G_t_max = F_et_gradient_val(Y_t_max)
                        cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max

                tau_f = .5*(tau_min + tau_max) # t_k satisfying both AW conditions
                Y_t_f = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f_t_f,G_t_f = F_et_gradient_val(Y_t_f)
                #cond_f_1 = (f_t_f <= f + rho1 * tau_f * DF_0) #for control
                #cond_f_2 = (.5 * np.trace(G_t_f.T @ np.linalg.inv(I + tau_f/2 * W) @ W @ (X + Y_t_f)) >= rho2* DF_0) #for control
                #print("Boucle d'itération:",i,"un tau qui va", tau_f, cond_f_1, cond_f_2 ) #for control 
                X = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f,G = F_et_gradient_val(X)
                #runn_scores.append(f)
                #if npmean(runn_scores[::-20])

            #print("Le score final est:",-f)
            return X,-f
        
        
    
        X,sc = algo_curv_search_train()
        X_val,sc_val = algo_curv_search_val()
        beta = fitBeta_train(X)
        cur_score = metric_val(X,beta)
        cur_accur = (cur_score/sc_val)*100
        QRTmet = metric_QRT(X,beta) # métrique QRT sur X_train
        testmet = metric_test(X,beta)
        met_QRT = metric_QRT_bch(X,beta) #métrique QRT benchmark
        
        #if l % 2 == 0:
        print( "Pour l'itération",l,"le score vaut",cur_accur,"la métrique QRT vaut",QRTmet, "et la métrique sur l'ensemble de validation vaut",testmet)

        if cur_accur > 10 and testmet > .045 and met_QRT > 0.14:
            print("Got a record:", cur_accur, "with QRT metric:",QRTmet, "at",val_locations,"for iteration",l, "for n = ",n,"with validation metric",testmet,"and benchmark QRT metric:", met_QRT )
            results.append([X,beta,QRTmet,cur_accur,testmet,met_QRT])
                
            
    # the last result is DA best !! 
    # This whole procedure is probably overfitting quite a lot, but what else... 
        
    return results
            
        
        
    
    
    
    
    
        
    
    

C:\Users\user\AppData\Local\Temp\ipykernel_10752\1448834574.py:22: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_QRT_reshape = pd.concat([ X_train_QRT.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Local\Temp\ipykernel_10752\1448834574.py:42: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_test_reshape = pd.concat([ X_test.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Local\Temp\ipykernel_10752\1448834574.py:51: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_Q_reshape = pd.concat([ X_train_Q.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


In [28]:

np.random.seed(432339)
results = []
Last_res = Cross_FV(n=4,Niter = 100)

C:\Users\user\AppData\Local\Temp\ipykernel_10752\1448834574.py:163: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Local\Temp\ipykernel_10752\1448834574.py:166: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\torch\_tensor.py:760: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Pour l'itération 0 le score vaut 15.284914360251742 la métrique QRT vaut 0.14671648167553072 et la métrique sur l'ensemble de validation vaut 0.050883561028491016
Pour l'itération 1 le score vaut 8.001055553881042 la métrique QRT vaut 0.15026715458173603 et la métrique sur l'ensemble de validation vaut 0.04630942246077914
Pour l'itération 2 le score vaut 6.361497249832899 la métrique QRT vaut 0.14900725544313628 et la métrique sur l'ensemble de validation vaut 0.04149026586071141
Pour l'itération 3 le score vaut 7.734723938398807 la métrique QRT vaut 0.15064977431581125 et la métrique sur l'ensemble de validation vaut 0.04789904262471444
Pour l'itération 4 le score vaut 23.32681144147807 la métrique QRT vaut 0.13552779898066503 et la métrique sur l'ensemble de validation vaut 0.03964939962193662
Pour l'itération 5 le score vaut 9.465531692748693 la métrique QRT vaut 0.15202513989736832 et la métrique sur l'ensemble de validation vaut 0.043924481632950436
Pour l'itération 6 le score vau

In [32]:
for j in range(len(Last_res)):
    if Last_res[j][4] > .05:
        print ("Le score courant est ",Last_res[j][4], " pour l'indice",j,"et une benchQRT",Last_res[j][4])

Le score courant est  0.058692348132444325  pour l'indice 1 et une benchQRT 0.058692348132444325
Le score courant est  0.05596258424538923  pour l'indice 4 et une benchQRT 0.05596258424538923
Le score courant est  0.05096157295935713  pour l'indice 6 et une benchQRT 0.05096157295935713
Le score courant est  0.05061017709255325  pour l'indice 7 et une benchQRT 0.05061017709255325
Le score courant est  0.05717549604076225  pour l'indice 8 et une benchQRT 0.05717549604076225
Le score courant est  0.053037824682429964  pour l'indice 11 et une benchQRT 0.053037824682429964


In [37]:
Last_res[11][5]

0.14298139167815258

In [57]:
for j in range(6):
    mach = metric_QRT(results[j][0],results[j][1]) 
    print("Le score sur la métrique QRT de l'élément",j+1,"est", mach,"et son score était",results[j][2])

Le score sur la métrique QRT de l'élément 1 est 0.14845519508497604 et son score était 24.041381853519727
Le score sur la métrique QRT de l'élément 2 est 0.1481737786630536 et son score était 25.07653179183571
Le score sur la métrique QRT de l'élément 3 est 0.14059700450100274 et son score était 29.848205544378008
Le score sur la métrique QRT de l'élément 4 est 0.14837132228630923 et son score était 34.54582577413859
Le score sur la métrique QRT de l'élément 5 est 0.14852832823499354 et son score était 41.12205494200732
Le score sur la métrique QRT de l'élément 6 est 0.14826523779211426 et son score était 44.46617267620694


In [38]:
QRT19 = Last_res[11][0]
BT19 = Last_res[11][1]

In [41]:
metric_QRT(QRT19,BT19)
metric_QRT_bch(QRT19,BT19)

0.14298139167815258

In [40]:
# On met le résultat dans un fichier du bon nom. 
reslong_QRT = parametersTransform(QRT19,BT19)
pd.DataFrame(reslong_QRT).to_csv('./exemple23_CVN0_3_n4.csv')

In [ ]:
 # renvoie n top scorers pour Xtrain
        def resultatsgrad(k = 5): 
            mat=[]
            for i in range(k):
                X = randomA()
                E,sc = algo_curv_search(X_start=X)
                mat.append(E)
            return mat    


        # On teste les n sur Xval; on renvoie la liste des scores sur X_val
        def top_scorer(E):
            res = []
            i=1
            for mat in E:
                beta = fitBeta(mat)
                score = metric_val(mat,beta)
                print("Le score pour la matrice",i,"est",score)
                res.append(score)
                i+=1
            j = np.argmax(res)
            print("Le top scorer est le no",k)
            return E[j]

In [53]:
len(results)

6

In [119]:
checkOrthonormality(EX6)

True

In [103]:
RES = []
def multCV(n=16,Niter = 40):
    the_res = []
    for k in range(47-n,47):
        curr_res = Cross_FV(k,Niter)
        print("On en est à n=",k,"et le meilleur score sur le val set est",curr_res[3])
        RES.append(curr_res)
        the_res.append(curr_res)
        print(" The best result for",k,"is",curr_res[2])
    return the_res
    

In [102]:
np.random.seed(7893)
FINAL_RES_CV = multCV(n=16,Niter = 60)

C:\Users\user\AppData\Local\Temp\ipykernel_8048\4282905070.py:24: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Local\Temp\ipykernel_8048\4282905070.py:27: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


Got a record: 0.041017143933057075 at [48 25 20  9 46 21 24 45 29 30 14 11 38 27 13 16  0 15 39  8  5 17  3 36
 33 42  4 37 35 41 28] for iteration 0 and for n =  31
Got a record: 0.04185237837247378 at [40 37 46 39  2 18 38 34  4 48  6 41 14 20 22 16 43  1 12  0 13  9 28 35
 31 24 47 49 25 32 15] for iteration 2 and for n =  31
Got a record: 0.044029479196612896 at [13 19 14 49 43  9 21  4 20 31 40 38  5 45 18 24 28 42 30 47  8 23 11  7
  0  3 48 12 37 25 26] for iteration 28 and for n =  31
Got a record: 0.044903724849602415 at [33  6 14 18  4  3  1 46 21 40  9 15 48  5 20 30 23  8 16 25 31 10 47 44
 17 11 22 36 41 32 19] for iteration 31 and for n =  31
Got a record: 0.04491005717701575 at [47 19 22  7 31 42 35 39  3 18  0  5 12 16 36 43 17 45 40  6 34 10 23 44
  9 33 20 28 11 25 29] for iteration 36 and for n =  31
Got a record: 0.05376020512109236 at [22 45 10 17 48  1  4 20 35 29 34 16 43 40 13 28  0  9 49 41 44 39 36 21
 27 46 31  3 15 38 33] for iteration 40 and for n =  31


IndexError: list index out of range

In [36]:
import pickle 
with open('Bch_22_11', 'wb') as f1:
    pickle.dump(A, f1)

In [67]:
EX1 = res[0]
Bt1 = res[1]

In [120]:
reslong_QRT6 = parametersTransform(EX6,Bt6)

In [121]:
pd.DataFrame(reslong_QRT6).to_csv('./exemple6_n=6.csv')

In [56]:
Last_res

[array([[-0.02679517,  0.08662926,  0.03063641, ...,  0.0140525 ,
          0.00048591, -0.00744233],
        [-0.11514198, -0.01465818, -0.019173  , ...,  0.0049001 ,
         -0.00443981, -0.06772024],
        [-0.06543031, -0.02364359, -0.06694202, ..., -0.1071562 ,
         -0.01239147,  0.04086212],
        ...,
        [-0.07298383,  0.0300135 ,  0.03702207, ...,  0.00301753,
          0.05553628, -0.09928898],
        [-0.04429377,  0.04963639,  0.00895816, ..., -0.05165773,
         -0.08020385, -0.01180377],
        [ 0.04241005,  0.04978732, -0.01228543, ...,  0.04607884,
          0.06654275, -0.1203566 ]]),
 array([-0.00382212,  0.06816777, -0.00100298,  0.00198185,  0.08871158,
         0.05637792,  0.02684837, -0.00907717, -0.02606879, -0.03190033]),
 0.09863197768225744]

In [57]:
XAV1 = Last_res[0]
beta1 = Last_res[1]

In [38]:
        
    n = 4
    
    def My_Own_CF(j):
            perm = np.random.permutation(range(50))
            valset = perm[:j]
            trainset = perm[j:]
            X_1 = X_train_Q.iloc[valset,]
            X_2 = X_train_Q.iloc[trainset,]
            Y_1 = Y_train_Q.iloc[valset,]
            Y_2 = Y_train_Q.iloc[trainset,]
            return X_2,Y_2,X_1,Y_1,valset

    X_train,Y_train,X_val,Y_val,val_locations = My_Own_CF(n)
    # Here we define our train and validation sets ! 


    X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
    X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

    X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
    X_val_reshape.columns = pd.Index(range(1,251), name='timeLag')

    # Tout le bazar stdt, pas besoin de se le retaper à chaque fois
    targets_t = torch.tensor(Y_train.T.stack().to_numpy().astype(np.float32)) # Le gros vecteur de pred 
    Y_true_t = torch.tensor(Y_train.to_numpy().astype(np.float32)) # La matrice à prédire en torch
    targets_val_t = torch.tensor(Y_val.T.stack().to_numpy().astype(np.float32)) # Le gros vecteur de pred du validation set  
    Y_true_val_t = torch.tensor(Y_val.to_numpy().astype(np.float32)) # La matrice à prédire du validation set en torch
    X_train_reshape_t = torch.tensor(X_train_reshape.to_numpy().astype(np.float32)) # X_train_reshape en torch
    X_val_reshape_t = torch.tensor(X_val_reshape.to_numpy().astype(np.float32)) # X_val_reshape en torch

C:\Users\user\AppData\Local\Temp\ipykernel_15976\606951516.py:17: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
C:\Users\user\AppData\Local\Temp\ipykernel_15976\606951516.py:20: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


In [39]:
    def F_et_gradient_val(A):

            A_t = torch.tensor(A.astype(np.float32),requires_grad = True) # On transforme l'entrée en tensor (on spécifie qu'on va différencier par rapport à A_t !!)
            predictors = X_val_reshape_t @ A_t # Les prédicteurs en tensor 
            beta_t = torch.inverse(predictors.T @ predictors)@predictors.T @targets_val_t # beta de l'équation normale en tensor
            Y_pred_t = (X_val_reshape_t @ A_t @ beta_t).resize(504,n).T #La matrice de prédiction en tensor
            f_A = torch.mean(sum(normalize(Y_pred_t,dim=0)*normalize(Y_true_val_t,dim=0)))
            f_A.backward() # On calcule le gradient
            A_output = -1*(A_t.grad).numpy() # Et voici le gradient en A !! C'est magique tellement c'est facile... 
            func_val = -1*f_A.detach().numpy() #la fonction de coût à minimiser évaluée en A 

            return func_val, A_output

    def algo_curv_search_val(X_start = randomA(),rho1 = 0.000001,rho2 = 0.5,eps = 0.001,j = 40): # la descente de gradient sur X_train
            
            start = time.time()
            X = X_start
            i,I = 0,np.eye(250)
            f,G = F_et_gradient_val(X) # f is f(X) et G the gradient of f at point X 
            tau_min = 0.00000001
            tau_max = 10
            #runn_scores = []

            while  (np.linalg.norm(G) >= eps) and (i <= j):
                i += 1
                if i % 5 == 0:
                    print("Nombre d'itérations:",i,"score actuel:", -f,"gradient actuel", np.linalg.norm(G))
                    print(time.ctime(time.time() - start)) # to keep track of the evolution of the algorithm
                tau_min = 0.0000001
                tau_max = 10
                W = G @ X.T - X @G.T #the skew symmetric matrix of the article _ we try to stick to the notations !  Remember, we need to update these at the end of the Armijo Wolfe conditions.          
                DF_0 = np.trace(G.T @ W @ X) #the directional derivative F'(Y(0)). 
                M_inv_min = np.linalg.inv(I + tau_min/2 * W) # Useful inverse matrices which we do not want to compute twice
                M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                f_t_max,G_t_max = F_et_gradient_val(Y_t_max)
                f_t_min,G_t_min = F_et_gradient_val(Y_t_min)
                cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max
                while (not cond_min) or (not cond_max) :
                    if not cond_min:
                        #print("Boucle d'itération:",i,"là c'est cond_min:", tau_min,tau_max) #pour check
                        if tau_min < 0.01:
                            tau_min = min(10*tau_min,tau_max - 0.0000000001)
                        else :
                            tau_min = min(1.05 * tau_min,tau_max - 0.000000001) 
                        #tau_min = .5*(tau_min + tau_max) # dichotomy is way too violent in this case... 
                        M_inv_min = np.linalg.inv(I + tau_min/2 * W)
                        Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                        f_t_min,G_t_min = F_et_gradient_val(Y_t_min)
                        cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
                    elif not cond_max:
                        #print("Boucle d'itération:",i,"là c'est cond_max:", tau_min,tau_max) #pour check
                        tau_max = max(tau_min+ 0.000000001,min(.95*tau_max,.5*(tau_max+tau_min))) # just to ensure that we always have tau_max > tau_min
                        # tau_max = .5*(tau_min + tau_max) same comment
                        M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                        Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                        f_t_max,G_t_max = F_et_gradient_val(Y_t_max)
                        cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max

                tau_f = .5*(tau_min + tau_max) # t_k satisfying both AW conditions
                Y_t_f = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f_t_f,G_t_f = F_et_gradient_val(Y_t_f)
                #cond_f_1 = (f_t_f <= f + rho1 * tau_f * DF_0) #for control
                #cond_f_2 = (.5 * np.trace(G_t_f.T @ np.linalg.inv(I + tau_f/2 * W) @ W @ (X + Y_t_f)) >= rho2* DF_0) #for control
                #print("Boucle d'itération:",i,"un tau qui va", tau_f, cond_f_1, cond_f_2 ) #for control 
                X = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
                f,G = F_et_gradient_val(X)
                #runn_scores.append(f)
                #if npmean(runn_scores[::-20])

            print("Le score final est:",-f)
            return X,-f
        

In [40]:
H = randomA()

In [41]:
F_et_gradient_val(H)

(-0.040650974959135056,
 array([[-5.83904679e-04, -2.01293500e-03,  1.37763540e-03, ...,
         -1.67436630e-03, -2.17501214e-03, -4.29129024e-04],
        [-5.27584972e-03, -6.57886593e-03,  4.60390141e-03, ...,
         -9.54798772e-04, -4.68726084e-03, -4.45055449e-03],
        [-1.01493555e-04,  2.01926799e-03, -1.36127695e-03, ...,
          2.60156998e-03,  2.67600059e-03, -1.91569736e-04],
        ...,
        [ 5.42101823e-03,  1.07930135e-02, -7.45559018e-03, ...,
          5.90529712e-03,  1.00153079e-02,  4.37387312e-03],
        [ 1.06903473e-02,  6.58742152e-04, -7.66952522e-04, ...,
         -1.35369692e-02, -6.83750538e-03,  9.64376330e-03],
        [-1.16239302e-03,  1.38287753e-04, -5.84391237e-05, ...,
          1.72821176e-03,  1.01405894e-03, -1.05896045e-03]], dtype=float32))

In [43]:
algo_curv_search_val()

Nombre d'itérations: 5 score actuel: 0.1940990686416626 gradient actuel 0.27995858
Thu Jan  1 01:00:04 1970
Nombre d'itérations: 10 score actuel: 0.2860577404499054 gradient actuel 0.21378195
Thu Jan  1 01:00:09 1970
Nombre d'itérations: 15 score actuel: 0.3244090676307678 gradient actuel 0.16680072
Thu Jan  1 01:00:14 1970
Nombre d'itérations: 20 score actuel: 0.33814141154289246 gradient actuel 0.1394944
Thu Jan  1 01:00:20 1970
Nombre d'itérations: 25 score actuel: 0.3450325131416321 gradient actuel 0.8247676
Thu Jan  1 01:00:28 1970
Nombre d'itérations: 30 score actuel: 0.34542131423950195 gradient actuel 0.31794584
Thu Jan  1 01:00:39 1970
Nombre d'itérations: 35 score actuel: 0.3456016182899475 gradient actuel 0.13448928
Thu Jan  1 01:00:50 1970
Nombre d'itérations: 40 score actuel: 0.34561094641685486 gradient actuel 0.5829864
Thu Jan  1 01:01:00 1970
Le score final est: 0.34637486934661865


(array([[-0.04761862,  0.09420199,  0.04286247, ..., -0.01264419,
         -0.02231062,  0.0222154 ],
        [ 0.02350539, -0.08008093,  0.04950161, ..., -0.0512574 ,
          0.05862464,  0.00989032],
        [-0.00478209, -0.03881939, -0.06256097, ...,  0.04758623,
          0.11898216, -0.08200416],
        ...,
        [ 0.02603782, -0.12641521, -0.01932352, ...,  0.02866886,
          0.00374781, -0.10437706],
        [ 0.02609229,  0.09916785, -0.04414824, ..., -0.02250776,
          0.03002496, -0.1047299 ],
        [-0.01258311,  0.05283789,  0.06962118, ..., -0.06002436,
         -0.13489481,  0.06619275]]),
 0.34637486934661865)